In [ ]:
pip install sklearn matplotlib

In [ ]:
pip install transformers

In [ ]:
import os
import json
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
model = AutoModelForSequenceClassification.from_pretrained("bucketresearch/politicalBiasBERT")

**Threshold for accuracy**

In [ ]:
import os
import json
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
from tqdm import tqdm
import numpy as np
from sklearn.metrics import accuracy_score

# Initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
model = AutoModelForSequenceClassification.from_pretrained("bucketresearch/politicalBiasBERT")

# Function to get model's prediction score
def get_model_score(text):
    tokenized_inputs = tokenizer(text, truncation=True, return_tensors='pt', max_length=512)
    inputs = {'input_ids': tokenized_inputs['input_ids'], 'attention_mask': tokenized_inputs['attention_mask']}
    outputs = model(**inputs)
    logits = outputs[0]
    probs = logits.softmax(dim=-1)[0].tolist()

    score = probs[0] * 1/6 + probs[1] * 3/6 + probs[2] * 5/6
    return score

# Load JSON files from the directory
data_dir = "./data"
all_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.json')]

scores = []
true_labels = []

for file in tqdm(all_files):
    with open(file, 'r') as f:
        article = json.load(f)

    title = article['title']
    body = " ".join([sent['sentence'] for sent in article['body']])
    full_text = title + " " + body

    true_labels.append(article['article-level-annotations']['stance'].lower())
    scores.append(get_model_score(full_text))

# Determine the best thresholds
left_center_thresholds = np.arange(0, 1, 0.01)
center_right_thresholds = np.arange(0, 1, 0.01)
best_accuracy = 0
best_left_center_threshold = 0
best_center_right_threshold = 0

for left_center_threshold in left_center_thresholds:
    for center_right_threshold in center_right_thresholds:
        if left_center_threshold < center_right_threshold:
            predicted_labels = ["left" if score <= left_center_threshold else "right" if score >= center_right_threshold else "center" for score in scores]
            accuracy = accuracy_score(true_labels, predicted_labels)
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_left_center_threshold = left_center_threshold
                best_center_right_threshold = center_right_threshold

print(f"Best Left-Center Threshold: {best_left_center_threshold}, Best Center-Right Threshold: {best_center_right_threshold}, Best Accuracy: {best_accuracy}")


100%|██████████| 300/300 [08:22<00:00,  1.67s/it]


Best Left-Center Threshold: 0.0, Best Center-Right Threshold: 0.7000000000000001, Best Accuracy: 0.5166666666666667


**Threshold for distribution**

In [ ]:

# Calculate the true distribution
true_distribution = {
    "left": true_labels.count("left") / len(true_labels),
    "center": true_labels.count("center") / len(true_labels),
    "right": true_labels.count("right") / len(true_labels)
}

# Function to calculate distribution difference
def distribution_difference(pred_labels, true_dist):
    pred_distribution = {
        "left": pred_labels.count("left") / len(pred_labels),
        "center": pred_labels.count("center") / len(pred_labels),
        "right": pred_labels.count("right") / len(pred_labels)
    }
    diff = sum(abs(true_dist[key] - pred_distribution[key]) for key in true_dist)
    return diff

# Determine the best thresholds
left_center_thresholds = np.arange(0, 1, 0.01)
center_right_thresholds = np.arange(0, 1, 0.01)
min_diff = float('inf')
best_left_center_threshold = 0
best_center_right_threshold = 0

for left_center_threshold in left_center_thresholds:
    for center_right_threshold in center_right_thresholds:
        if left_center_threshold < center_right_threshold:
            predicted_labels = ["left" if score <= left_center_threshold else "right" if score >= center_right_threshold else "center" for score in scores]
            diff = distribution_difference(predicted_labels, true_distribution)
            if diff < min_diff:
                min_diff = diff
                best_left_center_threshold = left_center_threshold
                best_center_right_threshold = center_right_threshold

print(f"Best Left-Center Threshold: {best_left_center_threshold}, Best Center-Right Threshold: {best_center_right_threshold}, Minimum Distribution Difference: {min_diff}")


Best Left-Center Threshold: 0.49, Best Center-Right Threshold: 0.51, Minimum Distribution Difference: 0.13333333333333333


In [ ]:
# Calculate true labels distribution
true_label_distribution = [true_labels.count('left') / len(true_labels),
                           true_labels.count('center') / len(true_labels),
                           true_labels.count('right') / len(true_labels)]

# Determine the best thresholds
left_center_thresholds = np.arange(0, 1, 0.01)
center_right_thresholds = np.arange(0, 1, 0.01)
best_mse = float('inf')
best_left_center_threshold = 0
best_center_right_threshold = 0

for left_center_threshold in left_center_thresholds:
    for center_right_threshold in center_right_thresholds:
        if left_center_threshold < center_right_threshold:
            predicted_labels = ["left" if score <= left_center_threshold else "right" if score >= center_right_threshold else "center" for score in scores]

            predicted_label_distribution = [predicted_labels.count('left') / len(predicted_labels),
                                            predicted_labels.count('center') / len(predicted_labels),
                                            predicted_labels.count('right') / len(predicted_labels)]

            mse = mean_squared_error(true_label_distribution, predicted_label_distribution)

            if mse < best_mse:
                best_mse = mse
                best_left_center_threshold = left_center_threshold
                best_center_right_threshold = center_right_threshold

print(f"Best Left-Center Threshold: {best_left_center_threshold}, Best Center-Right Threshold: {best_center_right_threshold}, Best MSE: {best_mse}")

Best Left-Center Threshold: 0.49, Best Center-Right Threshold: 0.51, Best MSE: 0.0023407407407407405


In [ ]:

# Evaluate with best thresholds
predicted_labels = ["left" if score <= best_left_center_threshold else "right" if score >= best_center_right_threshold else "center" for score in scores]
predicted_label_distribution = [predicted_labels.count('left') / len(predicted_labels),
                                predicted_labels.count('center') / len(predicted_labels),
                                predicted_labels.count('right') / len(predicted_labels)]
accuracy = accuracy_score(true_labels, predicted_labels)

print(f"Best Left-Center Threshold: {best_left_center_threshold}")
print(f"Best Center-Right Threshold: {best_center_right_threshold}")
print(f"True Label Distribution: {true_label_distribution}")
print(f"Predicted Label Distribution: {predicted_label_distribution}")
print(f"Accuracy: {accuracy}")

Best Left-Center Threshold: 0.49
Best Center-Right Threshold: 0.51
True Label Distribution: [0.27, 0.48, 0.25]
Predicted Label Distribution: [0.25, 0.5466666666666666, 0.20333333333333334]
Accuracy: 0.4533333333333333
